**First execute "Button to start", then "Button to run"**

In [ ]:
#@title **↓ Button to start**

#@markdown ## Input dataset:

#@markdown ---

data = "wine_trade_2021.csv" #@param ["wine_trade_2021.csv", "Download"] {type:"string"}

#@markdown ---

#------------------------INPUT----------------------------------------------
if data == "Download":
    from google.colab.files import upload
    print(f'↓ Upload dataset')
    fname = upload()
    fname = list(fname.keys())[0]
elif data == 'wine_trade_2021.csv':
    !curl -o wine_trade_2021.csv 'https://raw.githubusercontent.com/Jinedl/trade/main/wine_trade_2021.csv'
    fname = data
#---------------------------------------------------------------------------

#-------------------MODULES-&-DATA------------------------------------------
!pip install pycirclize
from pycirclize import Circos
from pycirclize.parser import Matrix
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

if '.csv' in fname:
    df_ = pd.read_csv(fname).dropna()
elif '.xlsx' in fname:
    df_ = pd.read_excel(fname).dropna()

df_.k = df_.k.astype(str)
df_.v = pd.to_numeric(df_.v, errors='coerce')
df_.q = pd.to_numeric(df_.q, errors='coerce')
df_ = df_.dropna().copy()

code_to_country = {
    4: 'Afghanistan',
    8: 'Albania',
    12: 'Algeria',
    16: 'American Samoa',
    20: 'Andorra',
    24: 'Angola',
    28: 'Antigua and Barbuda',
    31: 'Azerbaijan',
    32: 'Argentina',
    36: 'Australia',
    40: 'Austria',
    44: 'Bahamas',
    48: 'Bahrain',
    50: 'Bangladesh',
    51: 'Armenia',
    52: 'Barbados',
    56: 'Belgium',
    58: 'Belgium-Luxembourg',
    60: 'Bermuda',
    64: 'Bhutan',
    68: 'Bolivia (Plurinational State of)',
    70: 'Bosnia Herzegovina',
    72: 'Botswana',
    76: 'Brazil',
    84: 'Belize',
    86: 'Br. Indian Ocean Terr.',
    90: 'Solomon Isds',
    92: 'Br. Virgin Isds',
    96: 'Brunei Darussalam',
    100: 'Bulgaria',
    104: 'Myanmar',
    108: 'Burundi',
    112: 'Belarus',
    116: 'Cambodia',
    120: 'Cameroon',
    124: 'Canada',
    132: 'Cabo Verde',
    136: 'Cayman Isds',
    140: 'Central African Rep.',
    144: 'Sri Lanka',
    148: 'Chad',
    152: 'Chile',
    156: 'China',
    162: 'Christmas Isds',
    166: 'Cocos Isds',
    170: 'Colombia',
    174: 'Comoros',
    175: 'Mayotte',
    178: 'Congo',
    180: 'Dem. Rep. of the Congo',
    184: 'Cook Isds',
    188: 'Costa Rica',
    191: 'Croatia',
    192: 'Cuba',
    196: 'Cyprus',
    200: 'Czechoslovakia',
    203: 'Czechia',
    204: 'Benin',
    208: 'Denmark',
    212: 'Dominica',
    214: 'Dominican Rep.',
    218: 'Ecuador',
    222: 'El Salvador',
    226: 'Equatorial Guinea',
    231: 'Ethiopia',
    232: 'Eritrea',
    233: 'Estonia',
    238: 'Falkland Isds (Malvinas)',
    242: 'Fiji',
    246: 'Finland',
    251: 'France',
    258: 'French Polynesia',
    260: 'Fr. South Antarctic Terr.',
    262: 'Djibouti',
    266: 'Gabon',
    268: 'Georgia',
    270: 'Gambia',
    275: 'State of Palestine',
    276: 'Germany',
    278: 'Fmr Dem. Rep. of Germany',
    280: 'Fmr Fed. Rep. of Germany',
    288: 'Ghana',
    292: 'Gibraltar',
    296: 'Kiribati',
    300: 'Greece',
    304: 'Greenland',
    308: 'Grenada',
    316: 'Guam',
    320: 'Guatemala',
    324: 'Guinea',
    328: 'Guyana',
    332: 'Haiti',
    340: 'Honduras',
    344: 'China, Hong Kong SAR',
    348: 'Hungary',
    352: 'Iceland',
    360: 'Indonesia',
    364: 'Iran',
    368: 'Iraq',
    372: 'Ireland',
    376: 'Israel',
    380: 'Italy',
    384: "Côte d'Ivoire",
    388: 'Jamaica',
    392: 'Japan',
    398: 'Kazakhstan',
    400: 'Jordan',
    404: 'Kenya',
    408: "Dem. People's Rep. of Korea",
    410: 'Rep. of Korea',
    414: 'Kuwait',
    417: 'Kyrgyzstan',
    418: "Lao People's Dem. Rep.",
    422: 'Lebanon',
    426: 'Lesotho',
    428: 'Latvia',
    430: 'Liberia',
    434: 'Libya',
    440: 'Lithuania',
    442: 'Luxembourg',
    446: 'China, Macao SAR',
    450: 'Madagascar',
    454: 'Malawi',
    458: 'Malaysia',
    462: 'Maldives',
    466: 'Mali',
    470: 'Malta',
    478: 'Mauritania',
    480: 'Mauritius',
    484: 'Mexico',
    490: 'Other Asia, nes',
    496: 'Mongolia',
    498: 'Rep. of Moldova',
    499: 'Montenegro',
    500: 'Montserrat',
    504: 'Morocco',
    508: 'Mozambique',
    512: 'Oman',
    516: 'Namibia',
    520: 'Nauru',
    524: 'Nepal',
    528: 'Netherlands',
    530: 'Neth. Antilles',
    531: 'Curaçao',
    533: 'Aruba',
    534: 'Saint Maarten',
    535: 'Bonaire',
    540: 'New Caledonia',
    548: 'Vanuatu',
    554: 'New Zealand',
    558: 'Nicaragua',
    562: 'Niger',
    566: 'Nigeria',
    570: 'Niue',
    574: 'Norfolk Isds',
    579: 'Norway',
    580: 'N. Mariana Isds',
    583: 'FS Micronesia',
    584: 'Marshall Isds',
    585: 'Palau',
    586: 'Pakistan',
    591: 'Panama',
    598: 'Papua New Guinea',
    600: 'Paraguay',
    604: 'Peru',
    608: 'Philippines',
    612: 'Pitcairn',
    616: 'Poland',
    620: 'Portugal',
    624: 'Guinea-Bissau',
    626: 'Timor-Leste',
    634: 'Qatar',
    642: 'Romania',
    643: 'Russian Federation',
    646: 'Rwanda',
    652: 'Saint Barthélemy',
    654: 'Saint Helena',
    659: 'Saint Kitts and Nevis',
    660: 'Anguilla',
    662: 'Saint Lucia',
    666: 'Saint Pierre and Miquelon',
    670: 'Saint Vincent and the Grenadines',
    674: 'San Marino',
    678: 'Sao Tome and Principe',
    682: 'Saudi Arabia',
    686: 'Senegal',
    688: 'Serbia',
    690: 'Seychelles',
    694: 'Sierra Leone',
    697: 'Europe EFTA, nes',
    699: 'India',
    702: 'Singapore',
    703: 'Slovakia',
    704: 'Viet Nam',
    705: 'Slovenia',
    706: 'Somalia',
    710: 'South Africa',
    711: 'So. African Customs Union',
    716: 'Zimbabwe',
    724: 'Spain',
    728: 'South Sudan',
    729: 'Sudan',
    736: 'Fmr Sudan',
    740: 'Suriname',
    748: 'Swaziland',
    752: 'Sweden',
    757: 'Switzerland',
    760: 'Syria',
    762: 'Tajikistan',
    764: 'Thailand',
    768: 'Togo',
    772: 'Tokelau',
    776: 'Tonga',
    780: 'Trinidad and Tobago',
    784: 'United Arab Emirates',
    788: 'Tunisia',
    792: 'Turkey',
    795: 'Turkmenistan',
    796: 'Turks and Caicos Isds',
    798: 'Tuvalu',
    800: 'Uganda',
    804: 'Ukraine',
    807: 'TFYR of Macedonia',
    810: 'Fmr USSR',
    818: 'Egypt',
    826: 'United Kingdom',
    834: 'United Rep. of Tanzania',
    842: 'USA',
    849: 'US Misc. Pacific Isds',
    854: 'Burkina Faso',
    858: 'Uruguay',
    860: 'Uzbekistan',
    862: 'Venezuela',
    876: 'Wallis and Futuna Isds',
    882: 'Samoa',
    887: 'Yemen',
    891: 'Serbia and Montenegro',
    894: 'Zambia'
  }
#---------------------------------------------------------------------------

In [ ]:
#@title **↓ Button to run** { run: "auto" }

#@markdown ## Filter data:

#@markdown ---

#@markdown ### Year:
year = 2021 #@param {type:"integer"}

#@markdown ---

#@markdown ### Product:
#@markdown - 220410	Wine; sparkling
#@markdown - 220421	Wine; still, in containers holding 2 litres or less
#@markdown - 220422	Wine; still, in containers holding more than 2 litres but not more than 10 litres
#@markdown - 220429	Wine; still, in containers holding more than 10 litres
#@markdown - 220430	Grape must; n.e.c. in heading no. 2009, n.e.c. in item no. 2204.2
code_220410 = True #@param {type:"boolean"}
code_220421 = True #@param {type:"boolean"}
code_220422 = True #@param {type:"boolean"}
code_220429 = True #@param {type:"boolean"}
code_220430 = False #@param {type:"boolean"}
another_codes_separated_only_by_commas = "" #@param {type:"raw"}

#@markdown ---

#@markdown ### Countries:
Belgium = False #@param {type:"boolean"}
Bulgaria = False #@param {type:"boolean"}
Czechia = False #@param {type:"boolean"}
Denmark = False #@param {type:"boolean"}
Estonia = False #@param {type:"boolean"}
Ireland = False #@param {type:"boolean"}
Greece = False #@param {type:"boolean"}
Spain = True #@param {type:"boolean"}
France = True #@param {type:"boolean"}
Croatia = False #@param {type:"boolean"}
Italy = True #@param {type:"boolean"}
Cyprus = False #@param {type:"boolean"}
Latvia = False #@param {type:"boolean"}
Lithuania = False #@param {type:"boolean"}
Luxembourg = False #@param {type:"boolean"}
Hungary = False #@param {type:"boolean"}
Malta = False #@param {type:"boolean"}
Netherlands = False #@param {type:"boolean"}
Austria = False #@param {type:"boolean"}
Poland = False #@param {type:"boolean"}
Portugal = True #@param {type:"boolean"}
Romania = False #@param {type:"boolean"}
Slovakia = False #@param {type:"boolean"}
Slovenia = False #@param {type:"boolean"}
Finland = False #@param {type:"boolean"}
Sweden = False #@param {type:"boolean"}
Germany = True #@param {type:"boolean"}
United_Kingdom = False #@param {type:"boolean"}
Other = False #@param {type:"boolean"}
another_countries_separated_only_by_commas = "" #@param {type:"raw"}

#@markdown ---

#@markdown ### Numeric ranges:
min_value_thsd_dollars = 0 #@param {type:"number"}
max_value_thsd_dollars = 10000000 #@param {type:"number"}
#@markdown 1 litre of wine ~ 1 kg
min_quantity_thsd_kg = 0 #@param {type:"number"}
max_quantity_thsd_kg = 10000000 #@param {type:"number"}

#@markdown ---

#@markdown ### Visualisation:
parametr = "quantity" #@param ["value", "quantity"]

#@markdown ---

#------------------------INPUT----------------------------------------------
year = [year]

product = []
input_codes = [code_220410, code_220421, code_220422, code_220429, code_220430]
default_codes = ['220410', '220421', '220422', '220429', '220430']
for i, c in zip(input_codes, default_codes):
    if i:
        product.append(c)
if another_codes_separated_only_by_commas:
    product += another_codes_separated_only_by_commas.split(',')

country = []
input_countries = [Belgium, Bulgaria, Czechia, Denmark, Estonia, Ireland, Greece, Spain, France, Croatia, Italy, Cyprus, Latvia, Lithuania, Luxembourg, Hungary, Malta, Netherlands, Austria, Poland, Portugal, Romania, Slovenia, Slovakia, Finland, Sweden, Germany, United_Kingdom, Other]
default_countries = ['Belgium', 'Bulgaria', 'Czechia', 'Denmark', 'Estonia', 'Ireland', 'Greece', 'Spain', 'France', 'Croatia', 'Italy', 'Cyprus', 'Latvia', 'Lithuania', 'Luxembourg', 'Hungary', 'Malta', 'Netherlands', 'Austria', 'Poland', 'Portugal', 'Romania', 'Slovenia', 'Slovakia', 'Finland', 'Sweden', 'Germany', 'United Kingdom', 'Other']
for i, c in zip(input_countries, default_countries):
    if i:
        country.append(c)
if another_countries_separated_only_by_commas:
    country += another_countries_separated_only_by_commas.split(',')
#---------------------------------------------------------------------------

#-------------------------FILTER--------------------------------------------------
df = df_[(df_.t.isin(year)) & (df_.k.str.startswith(tuple(product)))].dropna().copy()

df.i = df.i.apply(lambda x: code_to_country[x] if code_to_country[x] in country else 'Other')
df.j = df.j.apply(lambda x: code_to_country[x] if code_to_country[x] in country else 'Other')

df = df[(df.i.isin(country)) & (df.j.isin(country)) & (df.i != df.j)][['i', 'j', 'q', 'v']].groupby(['i', 'j']).sum().reset_index().copy()

df = df[(min_quantity_thsd_kg <= df.q) & (df.q <= max_quantity_thsd_kg) &
        (min_value_thsd_dollars <= df.v) & (df.v <= max_value_thsd_dollars)].copy()
matrix = Matrix.parse_fromto_table(df[['i', 'j', parametr[0]]])
#---------------------------------------------------------------------------

#----------------------CHORD-DIAGRAM----------------------------------------
circos = Circos.initialize_from_matrix(
    matrix,
    space=3,
    r_lim=(93, 100),
    cmap="tab10",
    label_kws=dict(orientation="vertical"),
)
fig = circos.plotfig()
#---------------------------------------------------------------------------------

#-------------------------GRAPH--------------------------------------------------
#G = nx.from_pandas_edgelist(df, source='i', target='j', edge_attr=['v', 'q'], create_using=nx.DiGraph)
#
#pos = nx.spring_layout(G, seed=0)
#d = dict(G.degree(weight='q')) ##################################################################################
#
#plt.figure(figsize=(10, 10))
#
#nx.draw_networkx(G, pos=pos, with_labels=True,
#                 edgelist=[],
#                 connectionstyle='arc3, rad = 0.5',
#                 #**options,
#                 node_size=[v/300 for v in d.values()]) ##################################################################################
#
#for edge in G.edges(data='q'): ##################################################################################
#    nx.draw_networkx_edges(G, pos, edgelist=[edge],
#                           width=np.log(edge[2]), arrowsize=np.log(edge[2])*3,
#                           connectionstyle='arc3, rad = 0.5', node_size=6000)
#---------------------------------------------------------------------------------

#--------------------------------DATASET-------------------------------------------------
from google.colab import data_table
data_table.enable_dataframe_formatter()
df_formatted = df.copy()
df_formatted.columns = ['Exporter', 'Importer', 'Quantity_thsd_kg', 'Value_thsd_dollars']
df_formatted = df_formatted.style.set_properties(**{'text-align': 'right'}).format(precision=3, thousands=' ', subset=['Quantity_thsd_kg', 'Value_thsd_dollars'])
display(df_formatted)
#---------------------------------------------------------------------------

# **About app**

This project on GitHub:
https://github.com/Jinedl/trade